# 📝 Entrenamiento con Concatenación de Features como Texto

## 🎯 Enfoque Simple: Todo como Texto

Este notebook implementa el enfoque de **concatenación de features**, donde todas las variables (texto, edad, nivel educativo, desempeño) se unen en un solo texto que se alimenta al Transformer.

### Ejemplo de Transformación:

```
Input Original:
  texto_final: "vendedor de abarrotes"
  edad: 35
  nivel: 5
  desempeño: 1

Output Concatenado:
  "vendedor de abarrotes | edad: 35 años | educación: secundaria completa | desempeño: independiente"
```

### 📊 Comparación con Multimodal:

| Aspecto | Este Notebook (Concatenación) | Multimodal |
|---------|------------------------------|------------|
| Complejidad | ⭐ Simple | ⭐⭐⭐ Compleja |
| Código | ~200 líneas | ~500 líneas |
| Performance | Buena (baseline) | Excelente (+1-3%) |
| Edad como | Texto ("35 años") | Número (0.23 normalizado) |

---

**💡 Tip**: Usa este notebook como baseline rápido. Si necesitas mejor performance, usa el notebook multimodal.

---


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ============================================================================
# 📦 IMPORTS Y CONFIGURACIÓN INICIAL
# ============================================================================

import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report
)
from torch.utils.data import Dataset
import warnings
import logging
from datetime import datetime
import os
import json
import pickle

warnings.filterwarnings('ignore')

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

print("\n" + "="*80)
print(" IMPORTS COMPLETADOS")
print("="*80)
print(f"\n PyTorch version: {torch.__version__}")
print(f" CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f" GPU: {torch.cuda.get_device_name(0)}")
print("\n" + "="*80 + "\n")



 IMPORTS COMPLETADOS

 PyTorch version: 2.8.0+cu126
 CUDA disponible: True
 GPU: Tesla T4




In [ ]:
# ============================================================================
# ⚙️  CONFIGURACIÓN DEL EXPERIMENTO
# ============================================================================

class Config:
    """Configuración centralizada del experimento"""

    # ========================================================================
    # RUTAS DE DATOS
    # ========================================================================
    DATA_PATH = "/content/drive/MyDrive/classification_coding_open_ended_occupational_responses_ENAHO/CLEAN_DATA/BASE_LIMPIA_VF.parquet"  # 🔧 CAMBIAR AQUÍ

    # ========================================================================
    # COLUMNAS DEL DATASET
    # ========================================================================
    TEXT_COLUMN = "texto_final"         # Columna con descripción de ocupación
    EDAD_COLUMN = "p208a"              # Columna con edad (numérica)
    NIVEL_COLUMN = "p301a"             # Columna con nivel educativo (0-11)
    DESEMPENO_COLUMN = "p507"          # Columna con desempeño (0-3)
    TARGET_COLUMN = "p505r4"           # Columna target (código CIUO)

    # ========================================================================
    # MODELO BASE
    # ========================================================================
    MODEL_NAME = "FacebookAI/xlm-roberta-base"  # 🔧 Modelo recomendado

    # Alternativas (descomentar para usar):
    # MODEL_NAME = "dccuchile/bert-base-spanish-wwm-uncased"  # BETO
    # MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"           # RoBERTa-BNE
    # MODEL_NAME = "bert-base-multilingual-cased"             # mBERT

    # ========================================================================
    # FORMATO DE CONCATENACIÓN
    # ========================================================================
    INCLUDE_LABELS = True  # True: "edad: 35", False: "35"
    SEPARATOR = " , "      # Separador entre campos

    # Mapeos descriptivos para hacer el texto más legible
    NIVEL_EDUCATIVO_MAP = {
        1: "sin nivel educativo",
        2: "educación inicial",
        3: "primaria incompleta",
        4: "primaria completa",
        5: "secundaria incompleta",
        6: "secundaria completa",
        7: "superior no universitaria incompleta",
        8: "superior no universitaria completa",
        9: "superior universitaria incompleta",
        10: "superior universitaria completa",
        11: "maestría o doctorado",
        12: "básica especial"
    }

    DESEMPENO_MAP = {
        1: "empleador o patrono",
        2: "trabajador independiente",
        3: "empleado",
        4: 'obrero',
        5: "trabajador familiar no remunerado",
        6: 'trabajador del hogar',
        7: 'otro'
    }


    # ========================================================================
    # HIPERPARÁMETROS DE TOKENIZACIÓN
    # ========================================================================
    MAX_LENGTH = 128  # Longitud máxima de secuencia

    # ========================================================================
    # HIPERPARÁMETROS DE ENTRENAMIENTO
    # ========================================================================
    BATCH_SIZE = 16            # Tamaño del batch
    LEARNING_RATE = 2e-5       # Learning rate
    NUM_EPOCHS = 3             # Número de épocas
    WARMUP_STEPS = 500         # Pasos de warmup
    WEIGHT_DECAY = 0.01        # Weight decay
    EARLY_STOPPING_PATIENCE = 3  # Paciencia para early stopping

    # FILTRADO
    MIN_SAMPLES_PER_CLASS=10

    # ========================================================================
    # SPLITS DE DATOS
    # ========================================================================
    TEST_SIZE = 0.15    # 15% para test
    VAL_SIZE = 0.15     # 15% para validación
    RANDOM_STATE = 2025   # Semilla para reproducibilidad

    # ========================================================================
    # DIRECTORIOS DE SALIDA
    # ========================================================================
    OUTPUT_DIR = "./outputs_concatenado"
    EXPERIMENT_NAME = f"concat_{MODEL_NAME.split('/')[-1]}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    # Directorios específicos
    EXPERIMENT_DIR = os.path.join(OUTPUT_DIR, EXPERIMENT_NAME)
    CHECKPOINT_DIR = os.path.join(EXPERIMENT_DIR, "checkpoints")
    MODEL_SAVE_DIR = os.path.join(EXPERIMENT_DIR, "final_model")
    LOGS_DIR = os.path.join(EXPERIMENT_DIR, "logs")

    # ========================================================================
    # DEVICE
    # ========================================================================
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    @classmethod
    def create_directories(cls):
        """Crea los directorios necesarios"""
        os.makedirs(cls.EXPERIMENT_DIR, exist_ok=True)
        os.makedirs(cls.CHECKPOINT_DIR, exist_ok=True)
        os.makedirs(cls.MODEL_SAVE_DIR, exist_ok=True)
        os.makedirs(cls.LOGS_DIR, exist_ok=True)

    @classmethod
    def save_config(cls):
        """Guarda la configuración en JSON"""
        config_dict = {
            key: str(value) if not isinstance(value, (int, float, bool, str, dict)) else value
            for key, value in cls.__dict__.items()
            if not key.startswith('_') and not callable(value)
        }

        with open(os.path.join(cls.EXPERIMENT_DIR, 'config.json'), 'w') as f:
            json.dump(config_dict, f, indent=2)

# Inicializar configuración
config = Config()
config.create_directories()
config.save_config()

print("\n" + "="*80)
print("⚙️  CONFIGURACIÓN CARGADA")
print("="*80)
print(f"\n Experimento: {config.EXPERIMENT_NAME}")
print(f" Modelo: {config.MODEL_NAME}")
print(f" Device: {config.DEVICE}")
print(f" Formato: {'Con etiquetas' if config.INCLUDE_LABELS else 'Compacto'}")
print(f" Separador: '{config.SEPARATOR}'")
print(f" Max length: {config.MAX_LENGTH}")
print(f" Batch size: {config.BATCH_SIZE}")
print(f" Epochs: {config.NUM_EPOCHS}")
print(f"\n Output: {config.EXPERIMENT_DIR}")
print("\n" + "="*80 + "\n")



⚙️  CONFIGURACIÓN CARGADA

 Experimento: concat_xlm-roberta-base_20251112_085130
 Modelo: FacebookAI/xlm-roberta-base
 Device: cuda
 Formato: Con etiquetas
 Separador: ' , '
 Max length: 128
 Batch size: 16
 Epochs: 3

 Output: ./outputs_concatenado/concat_xlm-roberta-base_20251112_085130




In [ ]:
# ============================================================================
# FUNCIÓN DE CONCATENACIÓN DE FEATURES
# ============================================================================

def concatenate_features(
    texto: str,
    edad: int,
    nivel: int,
    desempeno: int,
    include_labels: bool = True,
    separator: str = " | "
) -> str:
    """
    Concatena todas las features en un solo texto enriquecido.

    Args:
        texto: Descripción textual de la ocupación
        edad: Edad de la persona (numérica)
        nivel: Nivel educativo (0-11)
        desempeno: Tipo de desempeño (0-3)
        include_labels: Si True, incluye etiquetas descriptivas
        separator: Separador entre componentes

    Returns:
        Texto concatenado con todas las features

    Ejemplos:
        >>> concatenate_features("vendedor", 35, 5, 1, True, " | ")
        'vendedor | edad: 35 años | educación: secundaria completa | desempeño: trabajador independiente'

        >>> concatenate_features("vendedor", 35, 5, 1, False, ", ")
        'vendedor, 35 años, secundaria completa, trabajador independiente'
    """
    # Limpiar texto base
    texto = str(texto).strip()

    # Iniciar con el texto original
    components = [texto]

    # Mapear valores categóricos a texto descriptivo
    nivel_desc = config.NIVEL_EDUCATIVO_MAP.get(int(nivel), "nivel desconocido")
    desemp_desc = config.DESEMPENO_MAP.get(int(desempeno), "desempeño desconocido")

    # Agregar componentes según el formato
    if include_labels:
        # Formato con etiquetas (más descriptivo)
        components.append(f"edad: {int(edad)} años")
        components.append(f"educación: {nivel_desc}")
        components.append(f"desempeño: {desemp_desc}")
    else:
        # Formato compacto (sin etiquetas)
        components.append(f"{int(edad)} años")
        components.append(nivel_desc)
        components.append(desemp_desc)

    # Unir todos los componentes
    return separator.join(components)


# ============================================================================
# PRUEBA DE LA FUNCIÓN
# ============================================================================

print("\n" + "="*80)
print("🔗 FUNCIÓN DE CONCATENACIÓN DEFINIDA")
print("="*80)
print("\n📝 Ejemplos de concatenación:\n")

# Ejemplo 1: Vendedor
ejemplo1 = concatenate_features(
    texto="vendedor de abarrotes en bodega",
    edad=35,
    nivel=5,
    desempeno=1,
    include_labels=config.INCLUDE_LABELS,
    separator=config.SEPARATOR
)
print("1️⃣  Vendedor (35 años, secundaria, independiente):")
print(f"   {ejemplo1}")

# Ejemplo 2: Profesor
ejemplo2 = concatenate_features(
    texto="profesor de matemáticas en colegio secundario",
    edad=42,
    nivel=9,
    desempeno=2,
    include_labels=config.INCLUDE_LABELS,
    separator=config.SEPARATOR
)
print("\n2️⃣  Profesor (42 años, universitaria, empleado):")
print(f"   {ejemplo2}")

# Ejemplo 3: Albañil
ejemplo3 = concatenate_features(
    texto="albañil en construcción",
    edad=50,
    nivel=3,
    desempeno=1,
    include_labels=config.INCLUDE_LABELS,
    separator=config.SEPARATOR
)
print("\n3️⃣  Albañil (50 años, primaria, independiente):")
print(f"   {ejemplo3}")

print("\n" + "="*80 + "\n")



🔗 FUNCIÓN DE CONCATENACIÓN DEFINIDA

📝 Ejemplos de concatenación:

1️⃣  Vendedor (35 años, secundaria, independiente):
   vendedor de abarrotes en bodega , edad: 35 años , educación: secundaria completa , desempeño: trabajador independiente

2️⃣  Profesor (42 años, universitaria, empleado):
   profesor de matemáticas en colegio secundario , edad: 42 años , educación: superior universitaria completa , desempeño: empleado

3️⃣  Albañil (50 años, primaria, independiente):
   albañil en construcción , edad: 50 años , educación: primaria completa , desempeño: trabajador independiente




In [ ]:
# ============================================================================
# 📂 CARGA Y PREPARACIÓN DE DATOS
# ============================================================================

print("\n" + "="*80)
print("📂 CARGANDO Y PREPARANDO DATOS")
print("="*80 + "\n")

# Cargar datos
logging.info(f"Cargando datos desde: {config.DATA_PATH}")
df = pd.read_parquet(config.DATA_PATH)

print(f"✅ Datos cargados exitosamente")
print(f"   Total de registros: {len(df):,}")
print(f"   Columnas: {list(df.columns)}")

# Validar que existan todas las columnas necesarias
required_cols = [
    config.TEXT_COLUMN,
    config.EDAD_COLUMN,
    config.NIVEL_COLUMN,
    config.DESEMPENO_COLUMN,
    config.TARGET_COLUMN
]

missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    raise ValueError(
        f"\n❌ Error: Faltan las siguientes columnas en el dataset:\n"
        f"   {', '.join(missing_cols)}\n\n"
        f"   Columnas disponibles: {', '.join(df.columns)}"
    )

print(f"\n✅ Todas las columnas requeridas están presentes")

# Verificar valores nulos
print(f"\n🔍 Verificando valores nulos:")
nulls = df[required_cols].isnull().sum()
for col, count in nulls.items():
    pct = count / len(df) * 100
    print(f"   {col}: {count:,} ({pct:.2f}%)")

# Filtrar registros con valores nulos
initial_size = len(df)
df = df.dropna(subset=required_cols)
removed = initial_size - len(df)

if removed > 0:
    print(f"\n⚠️  Se eliminaron {removed:,} registros con valores nulos")

print(f"\n✅ Dataset final: {len(df):,} registros ({len(df)/initial_size*100:.1f}% del original)")

print("\n" + "="*80 + "\n")

def filter_rare_classes(df):
    """Filtra clases con pocas muestras"""
    print(
        f"Filtrando clases con < {config.MIN_SAMPLES_PER_CLASS} muestras..."
        )

    class_counts = df[config.TARGET_COLUMN].value_counts()
    valid_classes = class_counts[class_counts >= config.MIN_SAMPLES_PER_CLASS].index
    df_filtered = df[df[config.TARGET_COLUMN].isin(valid_classes)].copy()

    print(
        f"   Clases originales: {len(class_counts):,}\n"
        f"   Clases mantenidas: {len(valid_classes):,}\n"
        f"✅ Registros después: {len(df_filtered):,}"
        )
    return df_filtered

df = filter_rare_classes(df)



📂 CARGANDO Y PREPARANDO DATOS

✅ Datos cargados exitosamente
   Total de registros: 316,022
   Columnas: ['anio', 'conglome', 'vivienda', 'hogar', 'codperso', 'p207', 'p207_label', 'p208a', 'p301a', 'p301a_label', 'p301a1', 'p301a1_label', 'p505r4', 'p505r4_label', 'txt505', 'txt505b', 'p506r4', 'p506r4_label', 'txt506', 'p507', 'p507_label', 'p510', 'p510_label', 'txt505_clean', 'txt505b_clean', 'txt506_clean', 'texto_final']

✅ Todas las columnas requeridas están presentes

🔍 Verificando valores nulos:
   texto_final: 0 (0.00%)
   p208a: 0 (0.00%)
   p301a: 79 (0.02%)
   p507: 0 (0.00%)
   p505r4: 0 (0.00%)

⚠️  Se eliminaron 79 registros con valores nulos

✅ Dataset final: 315,943 registros (100.0% del original)


Filtrando clases con < 10 muestras...
   Clases originales: 450
   Clases mantenidas: 357
✅ Registros después: 315,546


In [ ]:
# ============================================================================
# 🔗 CREAR COLUMNA CON TEXTO CONCATENADO
# ============================================================================

print("\n" + "="*80)
print("🔗 CREANDO TEXTO CONCATENADO CON TODAS LAS FEATURES")
print("="*80 + "\n")

logging.info("Concatenando features...")

# Aplicar función de concatenación
df['texto_concatenado'] = df.apply(
    lambda row: concatenate_features(
        texto=row[config.TEXT_COLUMN],
        edad=int(row[config.EDAD_COLUMN]),
        nivel=int(row[config.NIVEL_COLUMN]),
        desempeno=int(row[config.DESEMPENO_COLUMN]),
        include_labels=config.INCLUDE_LABELS,
        separator=config.SEPARATOR
    ),
    axis=1
)

print(f"✅ Columna 'texto_concatenado' creada exitosamente")

# Mostrar ejemplos
print(f"\n📝 Ejemplos de transformación (primeros 5 registros):\n")
print("-" * 80)

for i in range(min(5, len(df))):
    row = df.iloc[i]
    print(f"\n{i+1}. ORIGINAL:")
    print(f"   Texto: {row[config.TEXT_COLUMN]}")
    print(f"   Edad: {int(row[config.EDAD_COLUMN])} años")
    print(f"   Nivel: {int(row[config.NIVEL_COLUMN])} ({config.NIVEL_EDUCATIVO_MAP[int(row[config.NIVEL_COLUMN])]})")
    print(f"   Desempeño: {int(row[config.DESEMPENO_COLUMN])} ({config.DESEMPENO_MAP[int(row[config.DESEMPENO_COLUMN])]})")
    print(f"\n   CONCATENADO:")
    print(f"   {row['texto_concatenado']}")
    print("-" * 80)

# Análisis de longitud de texto
print(f"\n📏 ANÁLISIS DE LONGITUD (caracteres):\n")

original_lengths = df[config.TEXT_COLUMN].str.len()
concat_lengths = df['texto_concatenado'].str.len()

print(f"Texto Original:")
print(f"   Min: {original_lengths.min()}")
print(f"   Max: {original_lengths.max()}")
print(f"   Promedio: {original_lengths.mean():.1f}")
print(f"   Mediana: {original_lengths.median():.1f}")

print(f"\nTexto Concatenado:")
print(f"   Min: {concat_lengths.min()}")
print(f"   Max: {concat_lengths.max()}")
print(f"   Promedio: {concat_lengths.mean():.1f}")
print(f"   Mediana: {concat_lengths.median():.1f}")

print(f"\nIncremento:")
print(f"   Caracteres adicionales (promedio): +{concat_lengths.mean() - original_lengths.mean():.1f}")
print(f"   Factor de incremento: {concat_lengths.mean() / original_lengths.mean():.2f}x")

print("\n" + "="*80 + "\n")



🔗 CREANDO TEXTO CONCATENADO CON TODAS LAS FEATURES

✅ Columna 'texto_concatenado' creada exitosamente

📝 Ejemplos de transformación (primeros 5 registros):

--------------------------------------------------------------------------------

1. ORIGINAL:
   Texto: tejedora de chompa de lana en maquina de tejer tejer chompa de lana en maquina de tejer confección de chompa de lana en maquina de tejer en su vivienda
   Edad: 66 años
   Nivel: 4 (secundaria incompleta)
   Desempeño: 2 (empleado)

   CONCATENADO:
   tejedora de chompa de lana en maquina de tejer tejer chompa de lana en maquina de tejer confección de chompa de lana en maquina de tejer en su vivienda , edad: 66 años , educación: secundaria incompleta , desempeño: empleado
--------------------------------------------------------------------------------

2. ORIGINAL:
   Texto: tecnico administrativo digital documento manejar sistema hopsital essalud
   Edad: 47 años
   Nivel: 8 (superior universitaria incompleta)
   Desempeño: 3 

In [ ]:
# ============================================================================
# 🎯 CREAR MAPEOS DE LABELS Y SPLITS
# ============================================================================

print("\n" + "="*80)
print("🎯 CREANDO MAPEOS Y DIVIDIENDO DATOS")
print("="*80 + "\n")

# Crear mapeos label ↔ id
unique_labels = sorted(df[config.TARGET_COLUMN].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Crear columna con IDs
df['label_id'] = df[config.TARGET_COLUMN].map(label2id)

print(f"✅ Mapeos creados:")
print(f"   Número de clases únicas: {len(unique_labels)}")
print(f"   Ejemplos de mapeo:")
for i, (label, idx) in enumerate(list(label2id.items())[:5]):
    print(f"      {label} → {idx}")
if len(unique_labels) > 5:
    print(f"      ... ({len(unique_labels) - 5} más)")

# Distribución de clases
print(f"\n📊 Distribución de clases:")
class_dist = df['label_id'].value_counts().sort_index()
print(f"   Clase más frecuente: {class_dist.max():,} registros")
print(f"   Clase menos frecuente: {class_dist.min():,} registros")
print(f"   Promedio por clase: {class_dist.mean():.0f} registros")

# ============================================================================
# SPLITS ESTRATIFICADOS
# ============================================================================

print(f"\n🔪 Dividiendo en train/val/test...")

# Primer split: train vs (val + test)
train_df, temp_df = train_test_split(
    df,
    test_size=(config.TEST_SIZE + config.VAL_SIZE),
    random_state=config.RANDOM_STATE,
    stratify=df['label_id']
)

# Segundo split: val vs test
val_df, test_df = train_test_split(
    temp_df,
    test_size=config.TEST_SIZE / (config.TEST_SIZE + config.VAL_SIZE),
    random_state=config.RANDOM_STATE,
    stratify=temp_df['label_id']
)

print(f"\n✅ Splits creados (estratificados):")
print(f"   Train:      {len(train_df):>7,} registros ({len(train_df)/len(df)*100:>5.1f}%)")
print(f"   Validation: {len(val_df):>7,} registros ({len(val_df)/len(df)*100:>5.1f}%)")
print(f"   Test:       {len(test_df):>7,} registros ({len(test_df)/len(df)*100:>5.1f}%)")
print(f"   {'─'*40}")
print(f"   Total:      {len(df):>7,} registros")

# Verificar que las clases estén balanceadas en cada split
print(f"\n📊 Clases únicas por split:")
print(f"   Train: {train_df['label_id'].nunique()}")
print(f"   Val:   {val_df['label_id'].nunique()}")
print(f"   Test:  {test_df['label_id'].nunique()}")

print("\n" + "="*80 + "\n")



🎯 CREANDO MAPEOS Y DIVIDIENDO DATOS

✅ Mapeos creados:
   Número de clases únicas: 357
   Ejemplos de mapeo:
      0111 → 0
      0112 → 1
      0120 → 2
      0211 → 3
      0212 → 4
      ... (352 más)

📊 Distribución de clases:
   Clase más frecuente: 65,694 registros
   Clase menos frecuente: 10 registros
   Promedio por clase: 884 registros

🔪 Dividiendo en train/val/test...

✅ Splits creados (estratificados):
   Train:      220,882 registros ( 70.0%)
   Validation:  47,332 registros ( 15.0%)
   Test:        47,332 registros ( 15.0%)
   ────────────────────────────────────────
   Total:      315,546 registros

📊 Clases únicas por split:
   Train: 357
   Val:   357
   Test:  357




In [ ]:
# ============================================================================
# 📦 DATASET CLASS PARA TEXTO CONCATENADO
# ============================================================================

class ConcatenatedTextDataset(Dataset):
    """
    Dataset para manejar texto concatenado.

    Este dataset es más simple que el multimodal porque solo maneja
    una entrada de texto (aunque contiene toda la información).
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Args:
            texts: Lista de textos concatenados
            labels: Lista de labels (IDs numéricos)
            tokenizer: Tokenizer del modelo
            max_length: Longitud máxima de secuencia
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Obtener texto y label
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenizar
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✅ Clase ConcatenatedTextDataset definida")


✅ Clase ConcatenatedTextDataset definida


In [ ]:
# ============================================================================
# 🔤 CARGAR TOKENIZER Y CREAR DATASETS
# ============================================================================

print("\n" + "="*80)
print("🔤 CARGANDO TOKENIZER Y CREANDO DATASETS")
print("="*80 + "\n")

# Cargar tokenizer
logging.info(f"Cargando tokenizer: {config.MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

print(f"✅ Tokenizer cargado: {config.MODEL_NAME}")
print(f"   Tamaño del vocabulario: {len(tokenizer):,} tokens")
print(f"   Tokens especiales:")
print(f"      PAD: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"      UNK: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")
print(f"      CLS: {tokenizer.cls_token} (ID: {tokenizer.cls_token_id})")
print(f"      SEP: {tokenizer.sep_token} (ID: {tokenizer.sep_token_id})")

# Crear datasets
print(f"\n📦 Creando datasets...")

train_dataset = ConcatenatedTextDataset(
    texts=train_df['texto_concatenado'].values,
    labels=train_df['label_id'].values,
    tokenizer=tokenizer,
    max_length=config.MAX_LENGTH
)

val_dataset = ConcatenatedTextDataset(
    texts=val_df['texto_concatenado'].values,
    labels=val_df['label_id'].values,
    tokenizer=tokenizer,
    max_length=config.MAX_LENGTH
)

test_dataset = ConcatenatedTextDataset(
    texts=test_df['texto_concatenado'].values,
    labels=test_df['label_id'].values,
    tokenizer=tokenizer,
    max_length=config.MAX_LENGTH
)

print(f"\n✅ Datasets creados:")
print(f"   Train Dataset:      {len(train_dataset):,} ejemplos")
print(f"   Validation Dataset: {len(val_dataset):,} ejemplos")
print(f"   Test Dataset:       {len(test_dataset):,} ejemplos")

# Inspeccionar un ejemplo
print(f"\n🔍 Ejemplo de un elemento del dataset:")
sample = train_dataset[0]
print(f"   Input IDs shape: {sample['input_ids'].shape}")
print(f"   Attention mask shape: {sample['attention_mask'].shape}")
print(f"   Label: {sample['labels'].item()} (código: {id2label[sample['labels'].item()]})")
print(f"\n   Primeros 10 tokens:")
print(f"   IDs: {sample['input_ids'][:10].tolist()}")
decoded = tokenizer.decode(sample['input_ids'][:50])
print(f"   Texto: {decoded[:100]}...")

print("\n" + "="*80 + "\n")



🔤 CARGANDO TOKENIZER Y CREANDO DATASETS



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Tokenizer cargado: FacebookAI/xlm-roberta-base
   Tamaño del vocabulario: 250,002 tokens
   Tokens especiales:
      PAD: <pad> (ID: 1)
      UNK: <unk> (ID: 3)
      CLS: <s> (ID: 0)
      SEP: </s> (ID: 2)

📦 Creando datasets...

✅ Datasets creados:
   Train Dataset:      220,882 ejemplos
   Validation Dataset: 47,332 ejemplos
   Test Dataset:       47,332 ejemplos

🔍 Ejemplo de un elemento del dataset:
   Input IDs shape: torch.Size([128])
   Attention mask shape: torch.Size([128])
   Label: 194 (código: 5213)

   Primeros 10 tokens:
   IDs: [0, 173227, 11, 8, 28, 18908, 2759, 23778, 2721, 2512]
   Texto: <s> vendedora de esponjas escobillas guantes artículos de aseo hogar ofrecer desempolvar atender cli...




In [ ]:
# ============================================================================
# 📊 FUNCIONES DE MÉTRICAS
# ============================================================================

def compute_metrics(eval_pred):
    """
    Calcula métricas completas para evaluación.
    Incluye: accuracy, precision, recall, F1 (macro, micro, weighted)
    """
    predictions, labels = eval_pred

    # Obtener predicciones
    if predictions.ndim > 1:
        preds = np.argmax(predictions, axis=1)
    else:
        preds = predictions

    # Accuracy
    accuracy = accuracy_score(labels, preds)

    # Precision, Recall, F1 - Macro
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )

    # Precision, Recall, F1 - Micro
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        labels, preds, average='micro', zero_division=0
    )

    # Precision, Recall, F1 - Weighted
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'f1_weighted': f1_weighted,
        'precision_macro': precision_macro,
        'precision_micro': precision_micro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_micro': recall_micro,
        'recall_weighted': recall_weighted,
    }


def display_metrics(metrics, title="Métricas"):
    """Muestra las métricas de forma organizada"""
    print("\n" + "="*80)
    print(f"📊 {title.upper()}")
    print("="*80 + "\n")

    # Mostrar Loss si existe
    loss = (
        metrics.get('test_loss') or
        metrics.get('eval_loss') or
        metrics.get('loss', None)
    )
    if loss is not None:
        print(f"💥 LOSS: {loss:.4f}")

    # Accuracy
    acc = (
        metrics.get('test_accuracy') or
        metrics.get('eval_accuracy') or
        metrics.get('accuracy', 0)
    )
    print(f"🎯 ACCURACY: {acc:.4f}")
    print("\n" + "-"*80)

    # Tabla
    print(f"\n{'Métrica':<20} {'Macro':>12} {'Micro':>12} {'Weighted':>12}")
    print("-"*60)

    def get_m(name):
        return (
            metrics.get(f'test_{name}') or
            metrics.get(f'eval_{name}') or
            metrics.get(name, 0)
        )

    print(f"{'F1 Score':<20} {get_m('f1_macro'):>12.4f} {get_m('f1_micro'):>12.4f} {get_m('f1_weighted'):>12.4f}")
    print(f"{'Precision':<20} {get_m('precision_macro'):>12.4f} {get_m('precision_micro'):>12.4f} {get_m('precision_weighted'):>12.4f}")
    print(f"{'Recall':<20} {get_m('recall_macro'):>12.4f} {get_m('recall_micro'):>12.4f} {get_m('recall_weighted'):>12.4f}")

    print("\n" + "="*80 + "\n")


print("✅ Funciones de métricas cargadas (con loss incluido)")

✅ Funciones de métricas cargadas (con loss incluido)


In [ ]:
# ============================================================================
# 🤖 CARGAR MODELO Y CONFIGURAR ENTRENAMIENTO
# ============================================================================

print("\n" + "="*80)
print("🤖 CARGANDO MODELO Y CONFIGURANDO ENTRENAMIENTO")
print("="*80 + "\n")

# Cargar modelo
logging.info(f"Cargando modelo: {config.MODEL_NAME}")

model = AutoModelForSequenceClassification.from_pretrained(
    config.MODEL_NAME,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# Mover a device
model.to(config.DEVICE)

# Info del modelo
num_params = sum(p.numel() for p in model.parameters())
num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Modelo cargado: {config.MODEL_NAME}")
print(f"   Número de clases: {len(label2id)}")
print(f"   Parámetros totales: {num_params:,}")
print(f"   Parámetros entrenables: {num_trainable:,}")
print(f"   Device: {config.DEVICE}")

if torch.cuda.is_available():
    print(f"   Memoria GPU: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

# Configurar argumentos de entrenamiento
print(f"\n⚙️  Configurando argumentos de entrenamiento...")

training_args = TrainingArguments(
    output_dir=config.CHECKPOINT_DIR,
    logging_dir=config.LOGS_DIR,

    # Hiperparámetros
    learning_rate=config.LEARNING_RATE,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    num_train_epochs=config.NUM_EPOCHS,
    warmup_steps=config.WARMUP_STEPS,
    weight_decay=config.WEIGHT_DECAY,

    # Evaluación y guardado
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    greater_is_better=True,

    # Logging
    logging_steps=100,
    logging_strategy="steps",

    # Optimización
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=1,

    # Otros
    seed=config.RANDOM_STATE,
    report_to="none",
    disable_tqdm=False,
)

# Crear Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=config.EARLY_STOPPING_PATIENCE
        )
    ]
)

print(f"\n✅ Trainer configurado")
print(f"   Learning rate: {config.LEARNING_RATE}")
print(f"   Batch size: {config.BATCH_SIZE}")
print(f"   Epochs: {config.NUM_EPOCHS}")
print(f"   Early stopping patience: {config.EARLY_STOPPING_PATIENCE}")
print(f"   FP16: {training_args.fp16}")
print(f"   Métrica principal: f1_weighted")

print("\n" + "="*80 + "\n")



🤖 CARGANDO MODELO Y CONFIGURANDO ENTRENAMIENTO



model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modelo cargado: FacebookAI/xlm-roberta-base
   Número de clases: 357
   Parámetros totales: 278,318,181
   Parámetros entrenables: 278,318,181
   Device: cuda
   Memoria GPU: 1.11 GB

⚙️  Configurando argumentos de entrenamiento...

✅ Trainer configurado
   Learning rate: 2e-05
   Batch size: 16
   Epochs: 3
   Early stopping patience: 3
   FP16: True
   Métrica principal: f1_weighted




In [ ]:
# ============================================================================
# 🚀 ENTRENAMIENTO DEL MODELO
# ============================================================================

print("\n" + "="*80)
print("🚀 INICIANDO ENTRENAMIENTO")
print("="*80)
print(f"\n📝 Enfoque: CONCATENACIÓN DE FEATURES COMO TEXTO")
print(f"🤖 Modelo: {config.MODEL_NAME}")
print(f"📊 Datos de entrenamiento: {len(train_dataset):,} ejemplos")
print(f"📊 Datos de validación: {len(val_dataset):,} ejemplos")
print(f"🎯 Número de clases: {len(label2id)}")
print(f"\n⏱️  Esto puede tomar tiempo...")
print("="*80 + "\n")

try:
    start_time = datetime.now()
    logging.info("Iniciando entrenamiento...")

    # ENTRENAR
    train_result = trainer.train()

    end_time = datetime.now()
    training_time = end_time - start_time

    logging.info(f"Entrenamiento completado en {training_time}")

    print("\n" + "="*80)
    print("✅ ENTRENAMIENTO COMPLETADO")
    print("="*80)
    print(f"\n⏱️  Tiempo total: {training_time}")
    print(f"📉 Training loss: {train_result.training_loss:.4f}")

    # Evaluar en validation
    print("\n" + "-"*80)
    print("📊 Evaluando en conjunto de validación...")
    val_metrics = trainer.evaluate()
    display_metrics(val_metrics, "Métricas de Validación")

    print("="*80 + "\n")

except KeyboardInterrupt:
    print("\n⚠️  ENTRENAMIENTO INTERRUMPIDO POR EL USUARIO")
    raise

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR DURANTE EL ENTRENAMIENTO")
    print("="*80)
    print(f"\n{str(e)}\n")
    logging.error(f"Error durante entrenamiento: {str(e)}", exc_info=True)
    raise



🚀 INICIANDO ENTRENAMIENTO

📝 Enfoque: CONCATENACIÓN DE FEATURES COMO TEXTO
🤖 Modelo: FacebookAI/xlm-roberta-base
📊 Datos de entrenamiento: 220,882 ejemplos
📊 Datos de validación: 47,332 ejemplos
🎯 Número de clases: 357

⏱️  Esto puede tomar tiempo...



Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Precision Macro,Precision Micro,Precision Weighted,Recall Macro,Recall Micro,Recall Weighted
1,0.365700,0.362383,0.922040,0.422728,0.922040,0.911761,0.446585,0.922040,0.908964,0.431073,0.922040,0.922040
2,0.259100,0.291754,0.937970,0.524154,0.937970,0.932411,0.542102,0.937970,0.930588,0.536457,0.937970,0.937970
3,0.258300,0.272528,0.941836,0.562268,0.941836,0.937402,0.574126,0.941836,0.935642,0.572495,0.941836,0.941836



✅ ENTRENAMIENTO COMPLETADO

⏱️  Tiempo total: 2:02:30.245876
📉 Training loss: 0.4446

--------------------------------------------------------------------------------
📊 Evaluando en conjunto de validación...



📊 MÉTRICAS DE VALIDACIÓN

🎯 ACCURACY: 0.9418

--------------------------------------------------------------------------------

Métrica                     Macro        Micro     Weighted
------------------------------------------------------------
F1 Score                   0.5623       0.9418       0.9374
Precision                  0.5741       0.9418       0.9356
Recall                     0.5725       0.9418       0.9418





In [ ]:
# ============================================================================
# 🧪 EVALUACIÓN EN TEST SET
# ============================================================================

print("\n" + "="*80)
print("🧪 EVALUACIÓN EN TEST SET")
print("="*80 + "\n")

try:
    logging.info("Evaluando en test set...")

    # Obtener predicciones
    test_predictions = trainer.predict(test_dataset)
    test_metrics = test_predictions.metrics

    # Mostrar métricas
    display_metrics(test_metrics, "Métricas de Test (Evaluación Final)")

    # Guardar métricas
    metrics_file = os.path.join(config.EXPERIMENT_DIR, 'test_metrics.json')
    with open(metrics_file, 'w', encoding='utf-8') as f:
        json.dump(test_metrics, f, indent=2)

    print(f"💾 Métricas guardadas en: {metrics_file}")

    # Reporte de clasificación detallado
    print("\n" + "="*80)
    print("📈 REPORTE DE CLASIFICACIÓN DETALLADO")
    print("="*80 + "\n")

    y_pred = np.argmax(test_predictions.predictions, axis=1)
    y_true = test_predictions.label_ids

    # Nombres de clases
    target_names = [str(id2label[i]) for i in range(len(id2label))]

    class_report = classification_report(
        y_true,
        y_pred,
        target_names=target_names,
        zero_division=0,
        digits=4
    )

    print(class_report)

    # Guardar reporte
    report_file = os.path.join(config.EXPERIMENT_DIR, 'classification_report.txt')
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("REPORTE DE CLASIFICACIÓN - ENFOQUE DE CONCATENACIÓN\n")
        f.write("="*80 + "\n\n")
        f.write(f"Modelo: {config.MODEL_NAME}\n")
        f.write(f"Enfoque: Concatenación de features como texto\n")
        f.write(f"Formato: {'Con etiquetas' if config.INCLUDE_LABELS else 'Compacto'}\n")
        f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("\n" + "="*80 + "\n\n")
        f.write(class_report)

    print(f"\n💾 Reporte guardado en: {report_file}")

    # Análisis de errores
    print("\n" + "="*80)
    print("🔍 ANÁLISIS DE ERRORES")
    print("="*80 + "\n")

    incorrect_mask = y_pred != y_true
    num_incorrect = incorrect_mask.sum()

    print(f"Total de predicciones: {len(y_true):,}")
    print(f"Predicciones correctas: {(~incorrect_mask).sum():,} ({(~incorrect_mask).sum()/len(y_true)*100:.2f}%)")
    print(f"Predicciones incorrectas: {num_incorrect:,} ({num_incorrect/len(y_true)*100:.2f}%)")

    # Guardar casos incorrectos
    if num_incorrect > 0:
        errors_df = test_df[incorrect_mask].copy()
        errors_df['predicted_label'] = [id2label[pred] for pred in y_pred[incorrect_mask]]
        errors_df['true_label'] = [id2label[true] for true in y_true[incorrect_mask]]

        # Calcular confianza
        probs = torch.nn.functional.softmax(torch.tensor(test_predictions.predictions), dim=-1)
        max_probs = probs.max(dim=-1).values.numpy()
        errors_df['confidence'] = max_probs[incorrect_mask]

        errors_file = os.path.join(config.EXPERIMENT_DIR, 'error_analysis.csv')
        errors_df.to_csv(errors_file, index=False, encoding='utf-8')
        print(f"\n💾 Análisis de errores guardado en: {errors_file}")

    print("\n" + "="*80 + "\n")

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR EN LA EVALUACIÓN")
    print("="*80)
    print(f"\n{str(e)}\n")
    raise



🧪 EVALUACIÓN EN TEST SET




📊 MÉTRICAS DE TEST (EVALUACIÓN FINAL)

💥 LOSS: 0.2668
🎯 ACCURACY: 0.9416

--------------------------------------------------------------------------------

Métrica                     Macro        Micro     Weighted
------------------------------------------------------------
F1 Score                   0.5587       0.9416       0.9372
Precision                  0.5725       0.9416       0.9357
Recall                     0.5694       0.9416       0.9416


💾 Métricas guardadas en: ./outputs_concatenado/concat_xlm-roberta-base_20251112_085130/test_metrics.json

📈 REPORTE DE CLASIFICACIÓN DETALLADO

              precision    recall  f1-score   support

        0111     1.0000    0.3333    0.5000         3
        0112     0.0000    0.0000    0.0000         2
        0120     0.9000    1.0000    0.9474         9
        0211     0.7273    0.8889    0.8000         9
        0212     0.6667    0.8000    0.7273         5
        0213     0.0000    0.0000    0.0000         2
        0220     

In [ ]:
# ============================================================================
# 💾 GUARDADO DEL MODELO Y ARTEFACTOS
# ============================================================================

print("\n" + "="*80)
print("💾 GUARDANDO MODELO Y ARTEFACTOS")
print("="*80 + "\n")

try:
    # Guardar modelo y tokenizer
    logging.info(f"Guardando modelo en: {config.MODEL_SAVE_DIR}")

    model.save_pretrained(config.MODEL_SAVE_DIR)
    tokenizer.save_pretrained(config.MODEL_SAVE_DIR)

    print(f"✅ Modelo guardado en: {config.MODEL_SAVE_DIR}")

    # Guardar artefactos
    artifacts = {
        'label2id': label2id,
        'id2label': id2label,
        'num_labels': len(label2id),
        'model_name': config.MODEL_NAME,
        'approach': 'concatenation',
        'text_column': config.TEXT_COLUMN,
        'edad_column': config.EDAD_COLUMN,
        'nivel_column': config.NIVEL_COLUMN,
        'desempeno_column': config.DESEMPENO_COLUMN,
        'target_column': config.TARGET_COLUMN,
        'max_length': config.MAX_LENGTH,
        'include_labels': config.INCLUDE_LABELS,
        'separator': config.SEPARATOR,
        'nivel_educativo_map': config.NIVEL_EDUCATIVO_MAP,
        'desempeno_map': config.DESEMPENO_MAP,
        'test_metrics': test_metrics,
        'training_date': datetime.now().isoformat(),
    }

    artifacts_file = os.path.join(config.EXPERIMENT_DIR, 'artifacts.pkl')
    with open(artifacts_file, 'wb') as f:
        pickle.dump(artifacts, f)

    print(f"✅ Artefactos guardados en: {artifacts_file}")

    # Crear README
    readme_content = f"""# Modelo de Concatenación: {config.EXPERIMENT_NAME}

## Información del Modelo

- **Modelo Base**: {config.MODEL_NAME}
- **Enfoque**: Concatenación de features como texto
- **Número de Clases**: {len(label2id)}
- **Fecha de Entrenamiento**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Formato de Entrada

Este modelo usa CONCATENACIÓN de features:

```
Input Original:
  texto: "vendedor de abarrotes"
  edad: 35
  nivel: 5
  desempeño: 1

Texto Concatenado:
  "vendedor de abarrotes{config.SEPARATOR}edad: 35 años{config.SEPARATOR}educación: secundaria completa{config.SEPARATOR}desempeño: independiente"
```

## Configuración

- **Include labels**: {config.INCLUDE_LABELS}
- **Separator**: "{config.SEPARATOR}"
- **Max length**: {config.MAX_LENGTH}
- **Batch size**: {config.BATCH_SIZE}
- **Learning rate**: {config.LEARNING_RATE}
- **Epochs**: {config.NUM_EPOCHS}

## Resultados (Test Set)

- **Accuracy**: {test_metrics.get('test_accuracy', test_metrics.get('eval_accuracy', 0)):.4f}
- **F1 Weighted**: {test_metrics.get('test_f1_weighted', test_metrics.get('eval_f1_weighted', 0)):.4f}
- **F1 Macro**: {test_metrics.get('test_f1_macro', test_metrics.get('eval_f1_macro', 0)):.4f}

## Uso del Modelo

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pickle

# Cargar modelo y tokenizer
model = AutoModelForSequenceClassification.from_pretrained("{config.MODEL_SAVE_DIR}")
tokenizer = AutoTokenizer.from_pretrained("{config.MODEL_SAVE_DIR}")

# Cargar artefactos
with open("{artifacts_file}", 'rb') as f:
    artifacts = pickle.load(f)

# Preparar texto
def concatenate_features(texto, edad, nivel, desempeno):
    nivel_desc = artifacts['nivel_educativo_map'][nivel]
    desemp_desc = artifacts['desempeno_map'][desempeno]

    return f"{{texto}}{config.SEPARATOR}edad: {{edad}} años{config.SEPARATOR}educación: {{nivel_desc}}{config.SEPARATOR}desempeño: {{desemp_desc}}"

# Ejemplo
texto_concat = concatenate_features(
    texto="vendedor de abarrotes",
    edad=35,
    nivel=5,
    desempeno=1
)

# Tokenizar y predecir
inputs = tokenizer(texto_concat, return_tensors="pt", truncation=True, max_length={config.MAX_LENGTH})
outputs = model(**inputs)
predicted_class = outputs.logits.argmax(-1).item()
predicted_label = artifacts['id2label'][str(predicted_class)]

print(f"Predicción: {{predicted_label}}")
```

## Archivos

- `pytorch_model.bin`: Pesos del modelo
- `config.json`: Configuración del modelo
- `tokenizer.json`: Tokenizer
- `artifacts.pkl`: Mapeos y metadata
- `test_metrics.json`: Métricas de evaluación
- `classification_report.txt`: Reporte detallado
- `error_analysis.csv`: Análisis de errores

## Comparación con Multimodal

Este modelo usa concatenación simple (todas las features como texto).
Para mejor performance, considera el enfoque multimodal que preserva la información numérica.
"""

    readme_file = os.path.join(config.EXPERIMENT_DIR, 'README.md')
    with open(readme_file, 'w', encoding='utf-8') as f:
        f.write(readme_content)

    print(f"✅ README creado en: {readme_file}")

    print("\n" + "="*80)
    print("🎉 GUARDADO COMPLETADO")
    print("="*80 + "\n")

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR AL GUARDAR")
    print("="*80)
    print(f"\n{str(e)}\n")
    raise



💾 GUARDANDO MODELO Y ARTEFACTOS

✅ Modelo guardado en: ./outputs_concatenado/concat_xlm-roberta-base_20251112_085130/final_model
✅ Artefactos guardados en: ./outputs_concatenado/concat_xlm-roberta-base_20251112_085130/artifacts.pkl
✅ README creado en: ./outputs_concatenado/concat_xlm-roberta-base_20251112_085130/README.md

🎉 GUARDADO COMPLETADO



---

# 🎉 ¡ENTRENAMIENTO COMPLETADO!

## 📊 Modelo Entrenado con Concatenación

Has entrenado exitosamente un modelo usando el **enfoque de concatenación**, donde todas las features se unen en un solo texto:

```
"vendedor abarrotes | edad: 35 años | educación: secundaria completa | desempeño: independiente"
```

## 📈 Próximos Pasos

### 1. Comparar con Baseline

Compara este modelo con uno entrenado solo con texto (sin las features adicionales):

- Si mejora >1%: Las features adicionales ayudan ✅
- Si mejora <1%: Beneficio marginal ⚠️

### 2. Comparar con Multimodal

Para obtener el **mejor performance posible**, entrena también el modelo multimodal:

- Notebook: `training_multimodal_complete.ipynb`
- Esperado: +1-3% sobre concatenación
- Razón: Preserva información numérica de la edad

### 3. Análisis de Resultados

Revisa los archivos generados:

- `test_metrics.json`: Métricas completas
- `classification_report.txt`: Reporte por clase
- `error_analysis.csv`: Casos donde el modelo falló

## 💡 Ventajas y Limitaciones

### ✅ Ventajas de este Enfoque:

- **Simplicidad**: Código más simple y directo
- **Rápido**: Menos tiempo de desarrollo
- **Compatible**: Usa AutoModelForSequenceClassification estándar

### ⚠️ Limitaciones:

- **Pérdida de precisión numérica**: La edad "35" es solo texto, no número
- **Tokens extra**: Gasta tokens en etiquetas y formato
- **Menos flexible**: No puedes ajustar importancia de features

## 🔄 Workflow Completo

```
1. Solo Texto (baseline)
   ↓
2. Concatenación (este notebook)  ← ESTÁS AQUÍ
   ↓
3. Multimodal (mejor performance)
   ↓
4. Comparar y elegir el mejor
```

---

**¡Modelo con concatenación listo! 🚀**

Para mejor performance, prueba el notebook multimodal.
